In [1]:
# load packages
import xarray as xr
import numpy as np
import pandas as pd

## Data from van de Meer

> Marijn van der Meer, Sophie de Roda Husman, & Stef Lhermitte. (2023). Deep Learning Regional Climate Model Emulators: a comparison of two downscaling training frameworks [datasets] [Data set]. Zenodo. https://doi.org/10.5281/zenodo.7875882

- `MAR(ACCESS1-3)_monthly_SMB.nc`: RCM (MAR forced by ACCESS) smb values
  - variables: smb
  - temporal range: 1980 Jan - 2100 Dec (121 years)
  - temporal resolution: monthly
  - spatial resolution: 35km (in meters, thus 35,000 m)
  - projection: polar stereographic 3031
  - dimensionality: (1452, 148, 176)
- `MAR(ACCESS1-3)-stereographic_monthly_GCM_like.nc`: RCM (MAR forced by ACCESS) on the GCM grid
  - variables: SMB, .. SF, RF, SP, LWD, SWD, TT, TTP, QQP, PR, NW, EW
  - temporal range: 1980 Jan - 2100 Dec (121 years)
  - temporal resolution: monthly
  - spatial resolution: 206km (in meters, thus 206,000 m) in y direction, 68km (in meters, thus 68,000 m) in x direction
  - projection: polar stereographic 3031
  - dimensionality: (1452, 25, 90)
- `ACCESS1-3-stereographic_monthly_cleaned.nc`: 
  - variables: (no SMB included) huss, hurs, pr, tas, pd, rsds, tlds, vas, uas
  - temporal range: 1980 Jan - 2100 Dec (121 years)
  - temporal resolution: monthly
  - spatial resolution: 206km (in meters, thus 206,000 m) in y direction, 68km (in meters, thus 68,000 m) in x direction
  - projection: polar stereographic 3031
  - dimensionality: (1452, 25, 90)

Other details:
- cut to -40 lat

In [29]:
# define path to data
# files deleted again to free up space
full_path = "/Users/kimbente/Desktop/Bayesian Optimisation/BO-ANT-project/raw-data/Marvande_MAR(ACCESS1-3)/MAR(ACCESS1-3)_monthly_SMB.nc"
full_path = "/Users/kimbente/Desktop/Bayesian Optimisation/BO-ANT-project/raw-data/Marvande_MAR(ACCESS1-3)/MAR(ACCESS1-3)-stereographic_monthly_GCM_like.nc"
full_path = "/Users/kimbente/Desktop/Bayesian Optimisation/BO-ANT-project/raw-data/Marvande_MAR(ACCESS1-3)/ACCESS1-3-stereographic_monthly_cleaned.nc"

# open nc file as xarray dataset
ds = xr.open_dataset(full_path)
ds

<xarray.Dataset>
Dimensions:      (time: 1452, x: 90, y: 25)
Coordinates:
    spatial_ref  int64 ...
  * time         (time) datetime64[ns] 1980-01-31 1980-02-29 ... 2100-12-31
  * x            (x) float32 -2.992e+06 -2.924e+06 ... 2.992e+06 3.06e+06
  * y            (y) float32 -2.472e+06 -2.266e+06 ... 2.266e+06 2.472e+06
Data variables:
    huss         (time, y, x) float32 ...
    hurs         (time, y, x) float32 ...
    pr           (time, y, x) float64 ...
    tas          (time, y, x) float32 ...
    ps           (time, y, x) float32 ...
    rsds         (time, y, x) float32 ...
    rlds         (time, y, x) float32 ...
    vas          (time, y, x) float32 ...
    uas          (time, y, x) float32 ...
Attributes: (12/29)
    institution:            CSIRO (Commonwealth Scientific and Industrial Res...
    institute_id:           CSIRO-BOM
    experiment_id:          historical
    source:                 ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-p...
    model_id:               ACCESS1.3
    forcing:                GHG, Oz, SA, Sl, Vl, BC, OC, (GHG = CO2, N2O, CH4...
    ...                     ...
    title:                  ACCESS1-3 model output prepared for CMIP5 historical
    parent_experiment:      pre-industrial control
    modeling_realm:         atmos
    realization:            1
    cmor_version:           2.8.0
    crs:                    3031

In [30]:
# time is encoded as the last day of the month
ds.coords["time"]

<xarray.DataArray 'time' (time: 1452)>
array(['1980-01-31T00:00:00.000000000', '1980-02-29T00:00:00.000000000',
       '1980-03-31T00:00:00.000000000', ..., '2100-10-31T00:00:00.000000000',
       '2100-11-30T00:00:00.000000000', '2100-12-31T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
    spatial_ref  int64 ...
  * time         (time) datetime64[ns] 1980-01-31 1980-02-29 ... 2100-12-31

In [31]:
# coords are in meters in Polar Stereo
print(ds.coords["x"][0] - ds.coords["x"][1])
ds.coords["x"]

<xarray.DataArray 'x' ()>
array(-68000., dtype=float32)
Coordinates:
    spatial_ref  int64 0


<xarray.DataArray 'x' (x: 90)>
array([-2992000., -2924000., -2856000., -2788000., -2720000., -2652000.,
       -2584000., -2516000., -2448000., -2380000., -2312000., -2244000.,
       -2176000., -2108000., -2040000., -1972000., -1904000., -1836000.,
       -1768000., -1700000., -1632000., -1564000., -1496000., -1428000.,
       -1360000., -1292000., -1224000., -1156000., -1088000., -1020000.,
        -952000.,  -884000.,  -816000.,  -748000.,  -680000.,  -612000.,
        -544000.,  -476000.,  -408000.,  -340000.,  -272000.,  -204000.,
        -136000.,   -68000.,        0.,    68000.,   136000.,   204000.,
         272000.,   340000.,   408000.,   476000.,   544000.,   612000.,
         680000.,   748000.,   816000.,   884000.,   952000.,  1020000.,
        1088000.,  1156000.,  1224000.,  1292000.,  1360000.,  1428000.,
        1496000.,  1564000.,  1632000.,  1700000.,  1768000.,  1836000.,
        1904000.,  1972000.,  2040000.,  2108000.,  2176000.,  2244000.,
        2312000.,  2380000.,  2448000.,  2516000.,  2584000.,  2652000.,
        2720000.,  2788000.,  2856000.,  2924000.,  2992000.,  3060000.],
      dtype=float32)
Coordinates:
    spatial_ref  int64 ...
  * x            (x) float32 -2.992e+06 -2.924e+06 ... 2.992e+06 3.06e+06

In [32]:
ds.coords["y"]

<xarray.DataArray 'y' (y: 25)>
array([-2472000.  , -2266000.  , -2060000.  , -1853999.9 , -1647999.9 ,
       -1442000.  , -1236000.  , -1030000.  ,  -823999.94,  -618000.  ,
        -411999.97,  -205999.98,        0.  ,   205999.98,   411999.97,
         618000.  ,   823999.94,  1030000.  ,  1236000.  ,  1442000.  ,
        1647999.9 ,  1853999.9 ,  2060000.  ,  2266000.  ,  2472000.  ],
      dtype=float32)
Coordinates:
    spatial_ref  int64 ...
  * y            (y) float32 -2.472e+06 -2.266e+06 ... 2.266e+06 2.472e+06

In [33]:
ds.spatial_ref

<xarray.DataArray 'spatial_ref' ()>
[1 values with dtype=int64]
Coordinates:
    spatial_ref  int64 ...
Attributes: (12/16)
    crs_wkt:                                PROJCS["WGS 84 / Antarctic Polar ...
    semi_major_axis:                        6378137.0
    semi_minor_axis:                        6356752.314245179
    inverse_flattening:                     298.257223563
    reference_ellipsoid_name:               WGS 84
    longitude_of_prime_meridian:            0.0
    ...                                     ...
    grid_mapping_name:                      polar_stereographic
    standard_parallel:                      -71.0
    straight_vertical_longitude_from_pole:  0.0
    false_easting:                          0.0
    false_northing:                         0.0
    spatial_ref:                            PROJCS["WGS 84 / Antarctic Polar ...

In [37]:
print(121*12)

1452


## Data from Kittel

> Christoph Kittel. (2021). MAR projections of the Antarctic surface mass balance over the 21st century (ACCESS1.3) [Data set]. Zenodo. https://doi.org/10.5281/zenodo.4525735

- `MAR(ACCESS1-3)_monthly_SMB.nc`: RCM (MAR forced by ACCESS) smb values
  - variables: SF, RF, ME, SU (**RU and SMB missing?**)
  - temporal range: 1981 Jan - 2100 Dec (120 years)
    - each year
  - temporal resolution: **Daily**
  - spatial resolution: CORDEX_domain: ant-35km 35 in kilometers
  - projection: polar stereographic 3031
  - dimensionality: (365, 148, 176)
  - grid file in cdf

> Kittel, Christoph. (2021). Kittel et al. (2021), The Cryosphere : MAR and ESMs data [Data set]. Zenodo. https://doi.org/10.5281/zenodo.4459259

- `year-MAR_ACCESS1.3-1980-2100_zen.nc2`
   - temporal resolution: yearly


In [42]:
# yearly
# First file deleted again for storage
# full_path = "/Users/kimbente/Desktop/Bayesian Optimisation/BO-ANT-project/raw-data/Kittel_MAR/year-MAR_ACCESS1.3-1980-2100_zen.nc2"
full_path = "/Users/kimbente/Desktop/Bayesian Optimisation/BO-ANT-project/raw-data/Kittel_MAR/ME_ant-35km_ACCESS1.3_rcp8.5_r1i1p1_ULg-MAR311_v1_day_19810101-19811231.nc"

# open nc file as xarray dataset
ds = xr.open_dataset(full_path)

In [43]:
ds.coords["TIME"]

<xarray.DataArray 'TIME' (TIME: 365)>
array(['1981-01-01T12:00:00.000000000', '1981-01-02T12:00:00.000000000',
       '1981-01-03T12:00:00.000000000', ..., '1981-12-29T12:00:00.000000000',
       '1981-12-30T12:00:00.000000000', '1981-12-31T12:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * TIME     (TIME) datetime64[ns] 1981-01-01T12:00:00 ... 1981-12-31T12:00:00
Attributes:
    standard_name:  time
    long_name:      time
    bounds:         TIME_bnds
    axis:           T

In [46]:
ds.ME.values.shape

(365, 1, 148, 176)

In [50]:
ds.coords["X"]

<xarray.DataArray 'X' (X: 176)>
array([-3010., -2975., -2940., -2905., -2870., -2835., -2800., -2765., -2730.,
       -2695., -2660., -2625., -2590., -2555., -2520., -2485., -2450., -2415.,
       -2380., -2345., -2310., -2275., -2240., -2205., -2170., -2135., -2100.,
       -2065., -2030., -1995., -1960., -1925., -1890., -1855., -1820., -1785.,
       -1750., -1715., -1680., -1645., -1610., -1575., -1540., -1505., -1470.,
       -1435., -1400., -1365., -1330., -1295., -1260., -1225., -1190., -1155.,
       -1120., -1085., -1050., -1015.,  -980.,  -945.,  -910.,  -875.,  -840.,
        -805.,  -770.,  -735.,  -700.,  -665.,  -630.,  -595.,  -560.,  -525.,
        -490.,  -455.,  -420.,  -385.,  -350.,  -315.,  -280.,  -245.,  -210.,
        -175.,  -140.,  -105.,   -70.,   -35.,     0.,    35.,    70.,   105.,
         140.,   175.,   210.,   245.,   280.,   315.,   350.,   385.,   420.,
         455.,   490.,   525.,   560.,   595.,   630.,   665.,   700.,   735.,
         770.,   805.,   840.,   875.,   910.,   945.,   980.,  1015.,  1050.,
        1085.,  1120.,  1155.,  1190.,  1225.,  1260.,  1295.,  1330.,  1365.,
        1400.,  1435.,  1470.,  1505.,  1540.,  1575.,  1610.,  1645.,  1680.,
        1715.,  1750.,  1785.,  1820.,  1855.,  1890.,  1925.,  1960.,  1995.,
        2030.,  2065.,  2100.,  2135.,  2170.,  2205.,  2240.,  2275.,  2310.,
        2345.,  2380.,  2415.,  2450.,  2485.,  2520.,  2555.,  2590.,  2625.,
        2660.,  2695.,  2730.,  2765.,  2800.,  2835.,  2870.,  2905.,  2940.,
        2975.,  3010.,  3045.,  3080.,  3115.], dtype=float32)
Coordinates:
  * X        (X) float32 -3.01e+03 -2.975e+03 -2.94e+03 ... 3.08e+03 3.115e+03
Attributes:
    long_name:  x
    units:      km
    axis:       X

<xarray.Dataset>
Dimensions:    (TIME: 365, SECTOR1_1: 1, Y: 148, X: 176, bnds: 2)
Coordinates:
  * TIME       (TIME) datetime64[ns] 1981-01-01T12:00:00 ... 1981-12-31T12:00:00
  * X          (X) float32 -3.01e+03 -2.975e+03 -2.94e+03 ... 3.08e+03 3.115e+03
  * Y          (Y) float32 -2.555e+03 -2.52e+03 ... 2.555e+03 2.59e+03
  * SECTOR1_1  (SECTOR1_1) float32 1.0
Dimensions without coordinates: bnds
Data variables:
    ME         (TIME, SECTOR1_1, Y, X) float32 9.969e+36 9.969e+36 ... 9.969e+36
    TIME_bnds  (TIME, bnds) datetime64[ns] 1981-01-01 1981-01-02 ... 1982-01-01
Attributes: (12/21)
    CDI:                            Climate Data Interface version 1.9.7.1 (h...
    Conventions:                    CF-1.6
    history:                        Tue Feb  9 15:32:44 2021: ncap2 -O -s ME=...
    institute:                      University of Liege (Belgium)
    contact:                        ckittel@uliege.be
    NCO:                            netCDF Operators version 4.8.1 (Homepage ...
    ...                             ...
    frequency:                      day
    institute_id:                   ULg
    institution:                    University of Liege - Belgium
    model_id:                       ULg-MAR311
    product:                        output
    reference:                      http://www.climato.be